<a href="https://colab.research.google.com/github/ubiodee/SingularityNet/blob/main/Applied_Tags2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertModel
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the dataset
file_path = '/content/SNET_dataset.csv'
data = pd.read_csv(file_path)

# Prepare the input
meeting_notes = input("Enter meeting notes: ")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Tokenize the input meeting notes
encoded_inputs = tokenizer(meeting_notes, padding=True, truncation=True, return_tensors="pt")

# Get the predicted labels from the model
with torch.no_grad():
    output = model(**encoded_inputs)

# Get the predicted label indices
predicted_labels = torch.argmax(output.logits, dim=1).tolist()

# Extract unique tags from the dataset, handling NaN values
Y_flat = [tag for tag_list in data['Tags'].astype(str).str.split(',') for tag in tag_list if tag]

# Create a LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder to your list of unique tags
label_encoder.fit(Y_flat)

# Map the label indices to the corresponding tags
predicted_tags = [label_encoder.classes_[label] for label in predicted_labels]

# Find the most similar tags from the dataset
tag_embeddings = bert_model.embeddings.word_embeddings.weight[label_encoder.transform(label_encoder.classes_)]
tag_embeddings = tag_embeddings.detach().cpu().numpy()
predicted_label_embeddings = bert_model.embeddings.word_embeddings.weight[predicted_labels]
predicted_label_embeddings = predicted_label_embeddings.detach().cpu().numpy()
similarities = cosine_similarity(predicted_label_embeddings, tag_embeddings)

# Define the number of top tags to consider
top_n = 3

# Get the indices of the top N tags for each predicted label
top_n_indices = np.argsort(similarities, axis=1)[:, -top_n:]
top_n_tags = [[label_encoder.classes_[index] for index in indices[::-1]] for indices in top_n_indices]

# Print the top N similar tags
for i, tags in enumerate(top_n_tags):
    print(f"Predicted Label {i+1}: {predicted_tags[i]}")
    print(f"Top {top_n} Similar Tags: {tags}")
    print()


Enter meeting notes: This Workgroup focuses on creating educational project-based learning material for Dework on the Andamio platform in collaboration with Gimbalabs and Governance Guild


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Label 1:  AGIX
Top 3 Similar Tags: [' AGIX', ' Ethics', ' Staking']

